# ViT Fine-Tuning Demo

*Based on: https://huggingface.co/blog/fine-tune-vit*

Before running, download the [TrashNet dataset](https://github.com/garythung/trashnet/raw/master/data/dataset-resized.zip) and extract it to the 'dataset' folder.

In [ ]:
%pip install huggingface
%pip install datasets
%pip install pillow
%pip install transformers
%pip install scikit-learn
%pip install transformers[torch]
%pip install tensorboardX
# PyTorch + CUDA should be installed manually

## Dataset Processing

In [1]:
from datasets import load_dataset

ds = load_dataset("imagefolder", data_dir="dataset", split="train")
ds = ds.train_test_split(test_size=0.2, seed=512)

c:\Users\Dylan\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Resolving data files: 100%|██████████| 2527/2527 [00:00<00:00, 63860.21it/s]


In [2]:
from transformers import ViTImageProcessor, ViTConfig
from torchvision.transforms import Compose, RandomResizedCrop, RandomHorizontalFlip, ToTensor, Normalize, Resize, CenterCrop, ConvertImageDtype, InterpolationMode, ColorJitter
import torch

model_name_or_path = 'google/vit-base-patch16-224'
config = ViTConfig.from_pretrained(model_name_or_path)
processor = ViTImageProcessor.from_pretrained(model_name_or_path)

_train_transform = Compose([
    RandomResizedCrop(config.image_size, interpolation=InterpolationMode.BICUBIC, antialias=None),
    RandomHorizontalFlip(),
    ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    ToTensor(),
    ConvertImageDtype(torch.float),
    Normalize(
        mean=processor.image_mean,
        std=processor.image_std
    )
])

_val_transform = Compose([
    Resize(config.image_size, interpolation=InterpolationMode.BICUBIC, antialias=None),
    CenterCrop(config.image_size),
    ToTensor(),
    ConvertImageDtype(torch.float),
    Normalize(
        mean=processor.image_mean,
        std=processor.image_std
    )
])

def train_transform(example_batch):
    example_batch['pixel_values'] = [
        _train_transform(img.convert('RGB')) for img in example_batch['image']
    ]
    return example_batch

def val_transform(example_batch):
    example_batch['pixel_values'] = [
        _val_transform(img.convert('RGB')) for img in example_batch['image']
    ]
    return example_batch

ds['train'].set_transform(train_transform)
ds['test'].set_transform(val_transform)

## Training

In [3]:
import torch

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

In [4]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

C:\Users\Dylan\AppData\Local\Temp\ipykernel_2816\2425197347.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [5]:
from transformers import ViTForImageClassification

labels = ds['train'].features['label'].names

model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    ignore_mismatched_sizes=True,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([6]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([6, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from transformers import TrainingArguments
from transformers.trainer_utils import get_last_checkpoint
import os

if os.path.exists('./vit-base-trash'):
    last_checkpoint = get_last_checkpoint("./vit-base-trash")
else:
    last_checkpoint = None

training_args = TrainingArguments(
  output_dir="./vit-base-trash",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=32,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=20,
  learning_rate=3e-5, #3e-5
  save_total_limit=5,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
  resume_from_checkpoint=last_checkpoint
)

In [7]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    tokenizer=processor,
)

In [8]:
train_results = trainer.train(resume_from_checkpoint=last_checkpoint)
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

 99%|█████████▉| 4020/4064 [00:06<00:00, 1577.54it/s]

{'loss': 0.0526, 'learning_rate': 3.469488188976378e-07, 'epoch': 31.65}


 99%|█████████▉| 4040/4064 [00:11<00:00, 1577.54it/s]

{'loss': 0.0466, 'learning_rate': 1.9931102362204727e-07, 'epoch': 31.81}


100%|█████████▉| 4060/4064 [00:15<00:00, 1577.54it/s]

{'loss': 0.0514, 'learning_rate': 5.1673228346456694e-08, 'epoch': 31.97}


100%|██████████| 4064/4064 [00:16<00:00, 248.11it/s] 


{'train_runtime': 16.3819, 'train_samples_per_second': 3947.77, 'train_steps_per_second': 248.079, 'train_loss': 0.0007523451711777157, 'epoch': 32.0}
***** train metrics *****
  epoch                    =       32.0
  train_loss               =     0.0008
  train_runtime            = 0:00:16.38
  train_samples_per_second =    3947.77
  train_steps_per_second   =    248.079


In [11]:
metrics = trainer.evaluate(ds['test'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

100%|██████████| 64/64 [00:02<00:00, 23.98it/s]

***** eval metrics *****
  epoch                   =       32.0
  eval_accuracy           =     0.9506
  eval_loss               =     0.1887
  eval_runtime            = 0:00:02.89
  eval_samples_per_second =    175.029
  eval_steps_per_second   =     22.138
